In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""   Group model simulation with neurolib   -- Version 1.0
Last edit:  2023/11/13
Authors:    Leone, Riccardo (RL)
Notes:      - Running group-level simulations for CN WMH, MCI no/with WMH at a group level
            - Release notes:
                * Initial release
To do:      - 
Comments:   

Sources: 
"""

'   Model simulation with neurolib   -- Version 1.0\nLast edit:  2023/11/13\nAuthors:    Leone, Riccardo (RL)\nNotes:      - Running group-level simulations for CN WMH, MCI no/with WMH at a group level\n            - Release notes:\n                * Initial release\nTo do:      - \nComments:   \n\nSources: \n'

In [1]:
# %% Initial imports
import sys
import matplotlib.pyplot as plt
import neurolib.utils.functions as func
from neurolib.models.pheno_hopf import PhenoHopfModel
from neurolib.optimize.exploration import BoxSearch
from neurolib.utils import paths
from neurolib.utils import pypetUtils as pu
from neurolib.utils.parameterSpace import ParameterSpace

import filteredPowerSpectralDensity as filtPowSpectr
import my_functions as my_func

from petTOAD_parameter_setup import *
from petTOAD_setup import *

Getting the layout...
Done with the layout...
The following patients were discarded for having ROIs with all zeros: []
petTOAD Setup done!


In [3]:
import warnings
import numpy as np
from scipy import signal, stats
from numba import jit
import demean

discardOffset = 0
saveMatrix = False

def tril_indices_column(N, k=0):
    row_i, col_i = np.nonzero(
        np.tril(np.ones(N), k=k).T)  # Matlab works in column-major order, while Numpy works in row-major.
    Isubdiag = (col_i,
                row_i)  # Thus, I have to do this little trick: Transpose, generate the indices, and then "transpose" again...
    return Isubdiag


def triu_indices_column(N, k=0):
    row_i, col_i = np.nonzero(
        np.triu(np.ones(N), k=k).T)  # Matlab works in column-major order, while Numpy works in row-major.
    Isubdiag = (col_i,
                row_i)  # Thus, I have to do this little trick: Transpose, generate the indices, and then "transpose" again...
    return Isubdiag


# ==================================================================
# Computes the mean of the matrix
# ==================================================================
@jit(nopython=True)
def mean(x, axis=None):
    if axis == None:
        return np.sum(x, axis) / np.prod(x.shape)
    else:
        return np.sum(x, axis) / x.shape[axis]

@jit(nopython=True)
def adif(a, b):
    if np.abs(a - b) > np.pi:
        c = 2 * np.pi - np.abs(a - b)
    else:
        c = np.abs(a - b)
    return c

@jit(nopython=True)
def numba_PIM(phases, N, Tmax, dFC, PhIntMatr):
    T = np.arange(discardOffset, Tmax - discardOffset + 1)
    for t in T:
        for i in range(N):
            for j in range(i+1):
                dFC[i, j] = np.cos(adif(phases[i, t - 1], phases[j, t - 1]))
                dFC[j, i] = dFC[i, j]
        PhIntMatr[t - discardOffset] = dFC
    return PhIntMatr

def calc_PIM(ts, applyFilters=False, removeStrongArtefacts=False):  # Compute the Phase-Interaction Matrix of an input BOLD signal
    if not np.isnan(ts).any():  # No problems, go ahead!!!
        (N, Tmax) = ts.shape
        npattmax = Tmax - (2 * discardOffset - 1)  # calculates the size of phfcd matrix
        # Data structures we are going to need...
        phases = np.zeros((N, Tmax))
        dFC = np.zeros((N, N))
        # PhIntMatr = np.zeros((npattmax, int(N * (N - 1) / 2)))  # The int() is not needed, but... (see above)
        PhIntMatr = np.zeros((npattmax, N, N))
        # syncdata = np.zeros(npattmax)

        # Filters seem to be always applied...
        if applyFilters:
            ts_filt = BOLDFilters.BandPassFilter(ts, removeStrongArtefacts=removeStrongArtefacts)  # zero phase filter the data
        else:
            ts_filt = ts

        for n in range(N):
            Xanalytic = signal.hilbert(demean.demean(ts_filt[n, :]))
            phases[n, :] = np.angle(Xanalytic)

        PhIntMatr = numba_PIM(phases, N, Tmax, dFC, PhIntMatr)

    else:
        warnings.warn('############ Warning!!! PhaseInteractionMatrix.from_fMRI: NAN found ############')
        PhIntMatr = np.array([np.nan])
    # ======== sometimes we need to plot the matrix. To simplify the code, we save it here if needed...
    # if saveMatrix:
    #     import scipy.io as sio
    #     sio.savemat(save_file + '.mat', {name: PhIntMatr})
    return PhIntMatr

    
# ==================================================================
# numba_phFCD: convenience function to accelerate computations
# ==================================================================
@jit(nopython=True)
def numba_phFCD(phIntMatr_upTri, npattmax, size_kk3):
    phfcd = np.zeros((size_kk3))
    kk3 = 0

    for t in range(npattmax - 2):
        p1_sum = np.sum(phIntMatr_upTri[t:t + 3, :], axis=0)
        p1_norm = np.linalg.norm(p1_sum)
        for t2 in range(t + 1, npattmax - 2):
            p2_sum = np.sum(phIntMatr_upTri[t2:t2 + 3, :], axis=0)
            p2_norm = np.linalg.norm(p2_sum)

            dot_product = np.dot(p1_sum, p2_sum)
            phfcd[kk3] = dot_product / (p1_norm * p2_norm)
            kk3 += 1
    return phfcd


def phFCD(ts, applyFilters=False, removeStrongArtefacts=False):  # Compute the FCD of an input BOLD signal
    phIntMatr = calc_PIM(ts, applyFilters=applyFilters,
                                                 removeStrongArtefacts=removeStrongArtefacts)  # Compute the Phase-Interaction Matrix
    if not np.isnan(phIntMatr).any():  # No problems, go ahead!!!
        (N, Tmax) = ts.shape
        npattmax = Tmax - (2 * discardOffset - 1)  # calculates the size of phfcd vector
        size_kk3 = int((npattmax - 3) * (npattmax - 2) / 2)  # The int() is not needed because N*(N-1) is always even, but "it will produce an error in the future"...
        Isubdiag = tril_indices_column(N, k=-1)  # Indices of triangular lower part of matrix
        phIntMatr_upTri = np.zeros((npattmax, int(N * (N - 1) / 2)))  # The int() is not needed, but... (see above)
        for t in range(npattmax):
            phIntMatr_upTri[t,:] = phIntMatr[t][Isubdiag]
        phfcd = numba_phFCD(phIntMatr_upTri, npattmax, size_kk3,)
    else:
        warnings.warn('############ Warning!!! phFCD.from_fMRI: NAN found ############')
        phfcd = np.array([np.nan])
    # if saveMatrix:
    #     buildMatrixToSave(phfcd, npattmax - 2)
    return phfcd


In [4]:
def get_f_diff_group(group):
    # Get the timeseries for the chosen group
    group, timeseries = get_group_ts_for_freqs(group, all_fMRI_clean)
    f_diff = filtPowSpectr.filtPowSpetraMultipleSubjects(timeseries, TR)
    f_diff[np.where(f_diff == 0)] = np.mean(f_diff[np.where(f_diff != 0)])
    return f_diff


# Define the evaluation function for the subjectwise simulations
def evaluate(traj):
    # Get the trajectory from the search object
    model = search.getModelFromTraj(traj)
    # Create an empty list to store simulated BOLD
    bold_list = []
    # Initialize model with random initial conditions
    model.randomICs()
    # Run the model
    model.run(chunkwise=True, chunksize=60000, append=True)
    # Skip the first 120 secs to "warm up" the timeseries
    # Sample every 3 secs (3000/10 of downsampling = 300)
    ts = model.outputs.x[:, 12000::300]
    t = model.outputs.t[12000::300]
    # Apply the same z-score and detrending as the empirical data
    ts_filt = BOLDFilters.BandPassFilter(ts)
    # Append to the BOLD list
    bold_list.append(ts_filt)
    # We don't want the timeseries to have nans. If it has it means that the model "exploded"
    if not np.isnan([np.isnan(t).any() for t in bold_list]).any():
        print("No nans, good to go...")    
    else:
        print("There are some nans, aborting...")    

    # Create and save a dictionary with the resulting processed BOLD
    result_dict = {}
    result_dict["BOLD"] = np.array(bold_list).squeeze()
    result_dict["t"] = t
    search.saveToPypet(result_dict, traj)
    
def calculate_results_from_bolds(bold_arr, n_sim, n_parms, n_nodes):
    # Create a new array to store the FC and phFCD values for each parameter combination and simulation
    fc_array = np.zeros([n_sim, n_parms, n_nodes, n_nodes])
    phfcd_array = np.zeros([n_sim, n_parms, 18336])

    # Iterate over each element in the bold array
    for i in range(n_sim):
        for j in range(n_parms):
            print(
                f"Now calculating results from the {i+1}th simulation for parameter {j}..."
            )
            # Get the current timeseries
            timeseries = bold_arr[i, j].squeeze()

            # Recheck the timeseries for NaNs
            if np.isnan(timeseries).any():
                print("Simulation has some nans, aborting!")
                continue
            else:
                print("Simulation has no nans, good to go")
                print("Calculating FC..")
                fc_value = func.fc(timeseries)
                print("Calculating phFCD")
                phfcd_value = my_func.phFCD(timeseries)
                # Store the FC and phFCD value in the corresponding position in the arrays
                fc_array[i, j] = fc_value
                phfcd_array[i, j] = phfcd_value
    return fc_array, phfcd_array


def gather_results_from_repeated_simulations(
    group, grouplist, res_dir, filename
):
    # We get the trajectory names in the group simulation we want
    trajs = pu.getTrajectorynamesInFile(f"{res_dir}/{filename}")
    # Create a big list to store all results
    big_list = []
    # For every trajectory name we load the corresponding trajectory and store the associated bold for all runs
    for traj in trajs:
        traj_list = []
        tr = pu.loadPypetTrajectory(f"{res_dir}/{filename}", traj)
        run_names = tr.f_get_run_names()
        n_run = len(run_names)
        ns = range(n_run)
        for i in ns:
            r = pu.getRun(i, tr)
            traj_list.append(r["BOLD"])
        big_list.append(traj_list)
    # Convert the big list of BOLD for every run (combination of parameters) for every trajector (number of simulations)
    # to a numpy array
    bold_arr = np.array(big_list)
    n_sim = bold_arr.shape[0]
    n_parms = bold_arr.shape[1]
    # Calculate the arrays of FC and phFCD
    fc_array, phfcd_array = calculate_results_from_bolds(
        bold_arr, n_sim, n_parms, n_nodes
    )
    dict_group = {subj:all_fMRI_clean[subj] for subj in grouplist}
    # Get the group-averaged empirical FC and the concatenated empirical phFCD
    emp_fc, _, emp_phFCD = my_func.calc_and_save_group_stats(dict_group, Path(res_dir))
    # Get the average FC across the n simulations
    sim_fc = fc_array.mean(axis=0)
    print("Calculating fcs correlations...")
    fc_pearson = [func.matrix_correlation(row_sim_fc, emp_fc) for row_sim_fc in sim_fc]
    print("Calculating phFCDs KS distance...")
    phfcd_ks = [my_func.matrix_kolmogorov(phfcd_array[:, n].flatten(), emp_phFCD) for n in range(phfcd_array.shape[1])]
    print("Done calculating fitting measures")
    return fc_pearson, phfcd_ks


def create_df_results_het(fc_pearson, phfcd_ks, savedir, group):
    data = [
        [[(round(b, 3), round(w, 3)) for w in ws_het for b in bs_het], fc_pearson, phfcd_ks]
    ]
    columns = ["b_w", "fc_pearson", "phfcd_ks"]
    res_df = pd.DataFrame(data, columns=columns).explode(columns)
    res_df["b"], res_df["w"] = zip(*res_df.b_w)
    res_df = res_df.drop(columns=["b_w"])
    res_df.to_csv(f"{savedir}/df_results_heterogeneous_{group}.csv")

def create_df_results_sc_disconn(fc_pearson, phfcd_ks, savedir, group):
    data = [[fc_pearson, phfcd_ks]]
    columns = ["fc_pearson", "phfcd_ks"]
    res_df = pd.DataFrame(data, columns=columns).explode(columns)
    res_df.to_csv(f"{savedir}/df_results_disconn_{group}.csv")
    
def create_df_results_homo(parm_name, parms, fc_pearson, phfcd_ks, savedir, group):
    res_df = pd.DataFrame({parm_name: parms,
                           "fc_pearson": fc_pearson,
                           "phfcd_ks": phfcd_ks})
    res_df.to_csv(f"{savedir}/df_results_{group}_{parm_name}.csv")
    return res_df

In [5]:
def initialize_Hopf(fdiff):
    Dmat = np.zeros_like(sc)
    # Initialize the model (neurolib wants a Dmat to initialize the mode,
    model = PhenoHopfModel(Cmat=sc, Dmat=Dmat)
    # Empirical fmri is 193 timepoints at TR=3s (9.65 min) + 2 min of initial warm up of the timeseries
    model.params["duration"] = 11.65 * 60 * 1000
    model.params["signalV"] = 0
    model.params["dt"] = 0.1
    model.params["sampling_dt"] = 10.0
    model.params["sigma"] = 0.02
    model.params["w"] = 2 * np.pi * fdiff
    return model

def prepare_parms(group, model_type, G):
    
    if model_type == "homogeneous_a":
        # Define the parametere space to explore
        parameters = ParameterSpace(
            {
                "a":  [np.round(np.ones(n_nodes) * new_a, 3) for new_a in np.linspace(-0.1, -0.02, 81)],
                "K_gl": [G],
            },
            kind="grid",
        )
        filename = f"{group}_{model_type}_model.hdf"
        return parameters, filename
    
    elif model_type == "heterogeneous":
        node_damage =  get_group_node_spared(group) #now placeholder for testing, should become load_node_damage_group(group)
        parameters = ParameterSpace(
        {
            "a": [
                np.array([round(-0.02 + w * n + b, 5) for n in node_damage])
                for w in ws_het
                for b in bs_het
            ],
            "K_gl": [G],
        },
        kind="grid",
    )
        filename = f"{group}_{model_type}_model.hdf"
        return parameters, filename
    
    elif model_type == "homogeneous_G":
        
        if group == "CN_no_WMH":
            # dg otherwise the rightmost value is not included, sets how the delta G you want to explore
            # Since we explore a potentially much larger parameter space in CN_no_WMH we set the exploration
            # "granularity" to  a
            dg = 0.02
            parameters = ParameterSpace(
                {
                    "a":  [np.ones(n_nodes) * -0.02],
                    "K_gl": np.round(np.arange(0, G + dg, dg), 2),
                },
                kind="grid",
            )
            filename = f"{group}_{model_type}_model.hdf"
            return parameters, filename
            
        elif group == "MCI_no_WMH":
            return
        else:
            # Define the parametere space to explore
            dg = 0.01
            parameters = ParameterSpace(
                {
                    "a":  [np.ones(n_nodes) * -0.02],
                    "K_gl": np.round(np.arange(0, G + dg, dg), 2),
                },
                kind="grid",
            )
            filename = f"{group}_{model_type}_model.hdf"
            return parameters, filename

    elif model_type == "disconn":
        spared_sc = get_group_sc_wmh_weighted(group) #now placeholder for testing, should become disconn_sc = get_group_disconn(group)
        parameters = ParameterSpace(
        {
            "Cmat": [sc * (1 - spared_sc * w) for w in ws_disconn],
        },
        kind="grid",
    )
        filename = f"{group}_{model_type}_model.hdf"
        return parameters, filename

def create_df_results(model_type, parameters, fc_pearson, phfcd_ks, savedir, group):
    
    if model_type == "homogeneous_a":
        parm_name = "a"
        parms = parameters.a
        res_df = create_df_results_homo(parm_name, parms, fc_pearson, phfcd_ks, savedir, group)         
        return parm_name, res_df
    
    elif model_type == "heterogeneous":
        create_df_results_het(fc_pearson, phfcd_ks, savedir, group)
        return
            
    elif model_type == "homogeneous_G":
        parm_name = "K_gl"
        parms = parameters.K_gl
        res_df = create_df_results_homo(parm_name, parms, fc_pearson, phfcd_ks, savedir, group)
        if group == "CN_no_WMH":
            best_G = res_df[res_df["phfcd_ks"] == res_df["phfcd_ks"].min()]["K_gl"]
            best_G.to_csv(SIM_DIR / "best_G_CN_no_WMH")
        return parm_name, res_df

    elif model_type == "disconn":
        create_df_results_sc_disconn(savedir, fc_pearson, phfcd_ks, group)
        return

def save_plot_results_fitting_parm(parm_name, res_df, group):
    dict_x_lab = {"a": "Bifurcation parameter (a)",
                "K_gl": "Global coupling (G)"}
    if parm_name == "a":
        x = [res_df["a"][i][0] for i in range(res_df.shape[0])]
    else:
        x = res_df[parm_name]
    plt.figure()
    plt.plot(x, res_df["fc_pearson"], label="FC")
    plt.plot(x, res_df["phfcd_ks"], label="phFCD")
    plt.xlabel(dict_x_lab[parm_name])
    plt.ylabel(r"Pearson's $\rho$ / KS-distance")
    plt.legend()
    savedir = FIG_DIR / group
    if not Path.exists(savedir):
        Path.mkdir(savedir)
    plt.savefig(savedir/ f"{group}_{parm_name}_fitting.png")

def simulate_group(
    group, grouplist, fdiff, G, nsim, model_type
):
    global search
    # Set the directory where to save results
    savedir = str(SIM_DIR / group)
    paths.HDF_DIR = savedir
    print(
        f"Now performing the simulations for the {group} group, model: {model_type}"
    )
    model = initialize_Hopf(fdiff)
    parameters, filename = prepare_parms(group, model_type, G)
    for i in range(nsim):
        print(f"Starting simulations n°: {i+1}/{nsim}")
        # Initialize the search
        search = BoxSearch(
            model=model,
            evalFunction=evaluate,
            parameterSpace=parameters,
            filename=filename,
        )
        search.run(chunkwise=True, chunksize=60000, append=True)
    fc_pearson, phfcd_ks = gather_results_from_repeated_simulations(
        group, grouplist, savedir, filename,
    )
    if model_type not in ["heterogeneous", "disconn"]:
        parm_name, res_df = create_df_results(model_type, parameters, fc_pearson, phfcd_ks, savedir, group)
        save_plot_results_fitting_parm(parm_name, res_df, group)
        return res_df
    else:
        create_df_results(model_type, parameters, fc_pearson, phfcd_ks, savedir, group)
    print("Done!")
    
    


In [102]:
ws_min_disconn = 0.0
ws_max_disconn = 1.0
# Set the number of parameters you want your min-max interval to be split into 
n_ws_disconn = 101
# Create the final array with all the ws and bs you want to explore
ws_disconn = np.linspace(ws_min_disconn, ws_max_disconn, n_ws_disconn)
ws_disconn

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

In [6]:
group_dict = {"CN_no_WMH": CN_no_WMH, "MCI_no_WMH": MCI_no_WMH, "CN_WMH": CN_WMH, "MCI_WMH": MCI_WMH}
model_types_per_group = {
#"CN_no_WMH": ["homogeneous_G"],
#"MCI_no_WMH": ["homogeneous_a"],
"CN_WMH": ["disconn"], #["homogeneous_a", "homogeneous_G", "heterogeneous", "disconn"],
"MCI_WMH": ["disconn"] #["homogeneous_a", "homogeneous_G", "heterogeneous", "disconn"],
}


G = 3.5
nsim=1

for group_name, group_list in group_dict.items():
    for model_type in model_types_per_group[group_name]:
        fdiff = get_f_diff_group(group_list)
        if group_name == "CN_no_WMH":
            G = 3.5
        else:
            G = pd.read_csv(SIM_DIR / "best_G_CN_no_WMH")["K_gl"].to_list()    
        simulate_group(
                group_name, group_list, fdiff, G, nsim, model_type
            )

filtPowSpectraMultipleSubjects: subject 1 (of 63)
filtPowSpectraMultipleSubjects: subject 2 (of 63)
filtPowSpectraMultipleSubjects: subject 3 (of 63)
filtPowSpectraMultipleSubjects: subject 4 (of 63)
filtPowSpectraMultipleSubjects: subject 5 (of 63)
filtPowSpectraMultipleSubjects: subject 6 (of 63)
filtPowSpectraMultipleSubjects: subject 7 (of 63)
filtPowSpectraMultipleSubjects: subject 8 (of 63)
filtPowSpectraMultipleSubjects: subject 9 (of 63)
filtPowSpectraMultipleSubjects: subject 10 (of 63)
filtPowSpectraMultipleSubjects: subject 11 (of 63)
filtPowSpectraMultipleSubjects: subject 12 (of 63)
filtPowSpectraMultipleSubjects: subject 13 (of 63)
filtPowSpectraMultipleSubjects: subject 14 (of 63)
filtPowSpectraMultipleSubjects: subject 15 (of 63)
filtPowSpectraMultipleSubjects: subject 16 (of 63)
filtPowSpectraMultipleSubjects: subject 17 (of 63)
filtPowSpectraMultipleSubjects: subject 18 (of 63)
filtPowSpectraMultipleSubjects: subject 19 (of 63)
filtPowSpectraMultipleSubjects: subject 

AssertionError: /home/leoner/petTOAD/results/final_simulations/CN_no_WMH/CN_no_WMH_homogeneous_G_model.hdf does not exist!

In [ ]:
# ###############################################################
# #### Define the subject on which to perform the simulation ####
# ###############################################################
# id_subj = int(sys.argv[1]) - 1
# subj = subjs_to_sim[id_subj]
# n_subjs = len(subjs_to_sim)
# n_sim = 20
# best_G = 1.9

# ################################################################
# # Perform subject-wise simulations
# ################################################################
# if __name__ == "__main__":
#     # %% Get the frequencies for each group
#     f_diff_CN_no_wmh = get_f_diff_group(CN_no_WMH)
#     f_diff_CN_WMH = get_f_diff_group(CN_WMH)
#     f_diff_MCI_no_WMH = get_f_diff_group(MCI_no_WMH)
#     f_diff_MCI_WMH = get_f_diff_group(MCI_WMH)


#     print(f"SLURM ARRAY TASK: {id_subj} corresponds to subject {subj}")
#     print(f"We are going to do {n_sim} simulations for subject {subj}...")

#     random_conditions = [False, True]

#     for random_value in random_conditions:
#         if not random_value:
#             SIM_DIR_A = (
#                 SIM_DIR / f"a-weight_ws_{ws_min_a}-{ws_max_a}_bs_{bs_min_a}-{bs_max_a}"
#             )
#             SIM_DIR_G = (
#                 SIM_DIR / f"G-weight_ws_{ws_min_G}-{ws_max_G}_bs_{bs_min_G}-{bs_max_G}"
#             )
#             SIM_DIR_SC = SIM_DIR / f"sc_disconn"
#             SIM_DIR_HET = (
#                 SIM_DIR
#                 / f"heterogeneous_ws_{ws_min_het}-{ws_max_het}_bs_{bs_min_het}-{bs_max_het}"
#             )
#             wmh_dict = get_wmh_load_homogeneous(subjs)
#         else:
#             SIM_DIR_A = (
#                 SIM_DIR
#                 / f"a-weight_ws_{ws_min_a}-{ws_max_a}_bs_{bs_min_a}-{bs_max_a}_random"
#             )
#             SIM_DIR_G = (
#                 SIM_DIR
#                 / f"G-weight_ws_{ws_min_G}-{ws_max_G}_bs_{bs_min_G}-{bs_max_G}_random"
#             )
#             SIM_DIR_SC = SIM_DIR / f"sc_disconn"
#             SIM_DIR_HET = (
#                 SIM_DIR
#                 / f"heterogeneous_ws_{ws_min_het}-{ws_max_het}_bs_{bs_min_het}-{bs_max_het}_random"
#             )
#             wmh_dict = get_wmh_load_random(subjs)

#         sim_dir = [SIM_DIR_A, SIM_DIR_G, SIM_DIR_HET, SIM_DIR_SC]
#         for path in sim_dir:
#             if not Path.exists(path):
#                 Path.mkdir(path)

#         if subj in CN_WMH:
#             f_diff = f_diff_CN_WMH
#         elif subj in MCI_WMH:
#             f_diff = f_diff_MCI_WMH

#         simulate_homogeneous_model_a(
#             subj=subj,
#             f_diff=f_diff,
#             best_G=best_G,
#             wmh_dict=wmh_dict,
#             ws=ws_a,
#             bs=bs_a,
#             random_cond=random_value,
#             sim_dir=SIM_DIR_A,
#             nsim=n_sim,
#         )
#         simulate_homogeneous_model_G(
#             subj=subj,
#             f_diff=f_diff,
#             wmh_dict=wmh_dict,
#             ws=ws_G,
#             bs=bs_G,
#             random_cond=random_value,
#             sim_dir=SIM_DIR_G,
#             nsim=n_sim,
#         )

#         simulate_heterogeneous_model(
#             subj=subj,
#             cn_wmh_arr=CN_WMH,
#             mci_wmh_arr=MCI_WMH,
#             f_diff=f_diff,
#             best_G=best_G,
#             ws=ws_het,
#             bs=bs_het,
#             random_cond=random_value,
#             sim_dir=SIM_DIR_HET,
#             nsim=n_sim,
#         )
#         simulate_disconn_model(
#             subj=subj,
#             best_G=best_G,
#             f_diff=f_diff,
#             random_cond=random_value,
#             sim_dir=SIM_DIR_SC,
#             nsim=n_sim,
#         )
#         print("The end.")
